In [1]:
import numpy as np
import pandas as pd

otu_df = pd.read_csv('/home/esp/data_analyze/15.03.2024/data/init_data/Res1/OTU.txt', sep='\t')
hiera_blast = pd.read_csv('/home/esp/data_analyze/15.03.2024/data/init_data/Res1/hiera_BLAST.txt', sep='\t')
meta_map_init = pd.read_csv('/home/esp/data_analyze/15.03.2024/data/init_data/Res1/primary/in.map', sep='\t')
meta = pd.read_csv('/home/esp/data_analyze/15.03.2024/data/init_data/Res1/metadata_geo.csv',  sep='\t')
genus_df = pd.read_csv('/home/esp/data_analyze/15.03.2024/data/init_data/Res1/higherLvl/Genus.txt', sep='\t')

In [6]:
meta_map_df = meta_map_init.copy()
meta_map_df = meta_map_df.drop(columns=['SequencingRun'])
for column in meta_map_df.select_dtypes(include=['object']):  
    meta_map_df[column] = meta_map_df[column].str.replace('.extendedFrags.fastq.gz', '', regex=False)

meta_map_df['fastqFile'] = meta_map_df['fastqFile'].apply(lambda x: x.split('_')[0])
meta_map_df = meta_map_df.rename(columns={'fastqFile':'ID'})
meta_map_df['ID'] = pd.to_numeric(meta_map_df['ID'], errors='coerce')
meta_map_df.head()

,#SampleID,ID
0,SMPL0,10.0
1,SMPL1,12.0
2,SMPL10,66.0
3,SMPL100,12161.0
4,SMPL101,12162.0


In [7]:
meta.head()

,ID,GROUP
0,11593,1
1,11596,1
2,12001,2
3,12002,2
4,12004,2


In [13]:
# Совпадающие значения
matching_values = meta_map_df['ID'].isin(meta['ID'])
matched_values = meta_map_df['ID'][matching_values]

# Уникальные значения в meta_map_df
unique_in_meta_map_df = meta_map_df[~meta_map_df['ID'].isin(meta['ID'])]['ID']

# Уникальные значения в meta
unique_in_meta = meta[~meta['ID'].isin(meta_map_df['ID'])]['ID']

file = 'data/output_data/uniq_data.csv'
with open(file, 'w') as uniq_data_file:
    uniq_data_file.write(f"Совпадающие значения в обоих DataFrame ({len(matched_values.unique())}): {matched_values.unique()}\n\n"
                         f"Уникальные значения в in.map ({len(unique_in_meta_map_df.unique())}): {unique_in_meta_map_df.unique()}\n\n"
                         f"Уникальные значения в metadata ({len(unique_in_meta.unique())}): {unique_in_meta.unique()}\n\n"
                         f"Общее кол-во пропущенных данных: {len(unique_in_meta_map_df.unique()) + len(unique_in_meta.unique())}")

